In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, udf, upper
from datetime import datetime
from pyspark.sql.types import StringType

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 41, Finished, Available, Finished)

In [ ]:
# Create Spark session if not already created
spark = SparkSession.builder.getOrCreate()

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 42, Finished, Available, Finished)

In [ ]:
df = spark.read.table("sales_orders_bronze")

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 43, Finished, Available, Finished)

In [ ]:
# Impute missing values
df_imputed = df.withColumn(
    "CreatedDate", coalesce(col("CreatedDate"), col("LastModifiedDate"))
).withColumn(
    "LastModifiedDate", coalesce(col("LastModifiedDate"), col("CreatedDate"))
)

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 44, Finished, Available, Finished)

In [ ]:
df_imputed.show()

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 45, Finished, Available, Finished)

+-------+---------+------+-----------+----------------+--------+--------+
|SalesID|ProductID|   SKU|CreatedDate|LastModifiedDate|Quantity|UnitCost|
+-------+---------+------+-----------+----------------+--------+--------+
|      1|        1|sKU100| 2025/04/29|      2025-04-29|       5|  100.00|
|      2|        2|SKU200| 2025-04-30|      2025-04-30|      10|  140.00|
|      3|        3|sku300| 2025-04-28|      2025-04-28|      20|  150.00|
|      4|        1|sku100| 2025-04-27|      2025-04-27|       8|   40.00|
|      5|        4|SKU500| 2025-04-28|      2025-04-28|      15|  200.00|
|      6|        5|sku600| 2025-04-29|      2025-04-29|      30|  250.00|
|      7|        6|SKU700| 2025-04-27|      2025-04-27|      10|   90.00|
|      8|        2|sku200| 2025-04-26|      2025-04-26|      25|   80.00|
|      9|        3|SKU300| 2025-04-28|      2025-04-28|      12|  120.00|
|     10|        7|sku800| 2025/04/29|      2025-04-29|      18|  175.00|
+-------+---------+------+-----------+

In [ ]:
# UDF to convert dates in 'yyyy/MM/dd' format to 'yyyy-MM-dd'
def fix_date_format(date_str):
    if date_str is None:
        return None
    try:
        # If already correct format, return as is
        if "-" in date_str:
            datetime.strptime(date_str, "%Y-%m-%d")
            return date_str
        # If has slashes, convert it
        elif "/" in date_str:
            dt = datetime.strptime(date_str, "%Y/%m/%d")
            return dt.strftime("%Y-%m-%d")
    except ValueError:
        return None
    return date_str

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 46, Finished, Available, Finished)

In [ ]:
# Register UDF
fix_date_udf = udf(fix_date_format, StringType())

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 47, Finished, Available, Finished)

In [ ]:
# Apply date transformation
df_cleaned = df_imputed.withColumn("CreatedDate", fix_date_udf(col("CreatedDate")))

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 48, Finished, Available, Finished)

In [ ]:
# Transform SKU column to uppercase
df_transformed = df_cleaned.withColumn(
    "SKU", upper(col("SKU"))
)

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 49, Finished, Available, Finished)

In [ ]:
# Write the cleaned data to a new table
df_transformed.write.format("delta").mode("overwrite").saveAsTable("sales_order_silver")

StatementMeta(, 6e74f1a5-dbf4-479e-be4b-bb40b1282c37, 50, Finished, Available, Finished)